In [1]:
import pandas as pd
from openai import OpenAI
import random
import os
import re

In [2419]:
category = "figurative"
idiom = "make hit"
model ="gpt-4-turbo"

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="OpenAI Key",
)

In [2420]:
if not os.path.exists('response_data.csv'):
    pd.DataFrame(columns=["raw_response","category", "idiom", "type"]).to_csv('response_data.csv', index=False)


In [2421]:
df_structured = pd.DataFrame(columns=["submission", "category", "idiom", "type"])

**Zero-Shot**

In [2422]:
class ChatSession:
    def __init__(self, category, model=model):
        self.model = model
        system_message = f"Since English is your native language, you should use our language in rich and creative ways. Please avoid using similar structures in your sentences and craft sentences by using idioms correctly and with their {category} meanings. You should also avoid constructing sentences using personal names!"
        print(system_message)
        self.messages = [
            {"role": "system", "content": system_message}
        ]

    def get_completion(self, user_prompt):
        # Add the new user message
        self.messages.append({"role": "user", "content": user_prompt})
        
        # Make the API call
        response = client.chat.completions.create(
            model=self.model,
            messages=self.messages,
        )

        # Retrieve the response and append to messages to maintain context
        ai_response = response.choices[0].message.content.strip()
        self.messages.append({"role": "assistant", "content": ai_response})
        response_df = pd.DataFrame(columns=["raw_response", "idiom","type","category"])
        new_entry = {
            "raw_response": ai_response,
            "idiom": idiom,
            "type": type,
            "category":category
        }
        response_df = pd.concat([response_df, pd.DataFrame([new_entry])], ignore_index=True)
        response_df.to_csv('response_data.csv', mode='a', header=False, index=False)
        return ai_response

In [2423]:
chat_session = ChatSession(category=category)

Since English is your native language, you should use our language in rich and creative ways. Please avoid using similar structures in your sentences and craft sentences by using idioms correctly and with their figurative meanings. You should also avoid constructing sentences using personal names!


In [2424]:
type = "Zero-shot"
step = 12

In [2425]:
prompt = f"""
"{idiom}" is an English idiom. We can use this idiom in both literal and figurative senses. Please write five example sentences using this idiom in the {category} sense. In your response, return only the sentences.
"""
print(prompt)
print("\n")
responses = []
for i in range(step):
    result = chat_session.get_completion(prompt)
    responses.append(result)


"make hit" is an English idiom. We can use this idiom in both literal and figurative senses. Please write five example sentences using this idiom in the figurative sense. In your response, return only the sentences.





In [2426]:
responses

["1. The new musician hoped his fresh take on jazz would really make a hit with the underground music scene.\n2. Trying innovative marketing strategies can sometimes make a hit, catching the competitor's attention.\n3. When they launched their new software update, it really made a hit and user subscriptions doubled.\n4. The novelist's latest thriller made a hit and quickly soared to the top of the bestseller charts.\n5. She made a hit at the conference by introducing her unique approach to sustainable design.",
 "1. The mobile app made a hit in the tech community, with thousands downloading it on the first day.\n2. Her speech at the gala made a hit, leaving the audience inspired and motivated.\n3. The chef's new fusion dish made a hit among the food critics, earning rave reviews.\n4. Their innovative advertising campaign made a hit, significantly boosting product recognition.\n5. His revolutionary approach to teaching made a hit with both students and parents, enhancing engagement and 

In [2427]:
def parse_and_structure_data(raw_responses_list, idiom, category, response_type):
    # Initialize the list to store structured data
    structured_data = []
    # Iterate through each block of responses
    for block in raw_responses_list:
        # Split each block into individual sentences
        sentences = block.split('\n')
        # Remove the numbering from each sentence and strip leading/trailing whitespace
        clean_sentences = [sentence.split('. ', 1)[-1].strip() for sentence in sentences]
        # For each sentence, append a tuple with the structured data
        for sentence in clean_sentences:
            structured_data.append((sentence, category, idiom, response_type))
    return structured_data

In [2428]:
structured_data = parse_and_structure_data(responses, idiom, category, type)

new_df= pd.DataFrame(structured_data, columns=["submission", "category", "idiom", "type"])
new_df

,submission,category,idiom,type
0,The new musician hoped his fresh take on jazz ...,figurative,make hit,Zero-shot
1,Trying innovative marketing strategies can som...,figurative,make hit,Zero-shot
2,"When they launched their new software update, ...",figurative,make hit,Zero-shot
3,The novelist's latest thriller made a hit and ...,figurative,make hit,Zero-shot
4,She made a hit at the conference by introducin...,figurative,make hit,Zero-shot
5,The mobile app made a hit in the tech communit...,figurative,make hit,Zero-shot
6,"Her speech at the gala made a hit, leaving the...",figurative,make hit,Zero-shot
7,The chef's new fusion dish made a hit among th...,figurative,make hit,Zero-shot
8,Their innovative advertising campaign made a h...,figurative,make hit,Zero-shot
9,His revolutionary approach to teaching made a ...,figurative,make hit,Zero-shot


In [2429]:
new_df = new_df[new_df['submission']!=""]
new_df.reset_index(drop=True,inplace=True)
new_df


,submission,category,idiom,type
0,The new musician hoped his fresh take on jazz ...,figurative,make hit,Zero-shot
1,Trying innovative marketing strategies can som...,figurative,make hit,Zero-shot
2,"When they launched their new software update, ...",figurative,make hit,Zero-shot
3,The novelist's latest thriller made a hit and ...,figurative,make hit,Zero-shot
4,She made a hit at the conference by introducin...,figurative,make hit,Zero-shot
5,The mobile app made a hit in the tech communit...,figurative,make hit,Zero-shot
6,"Her speech at the gala made a hit, leaving the...",figurative,make hit,Zero-shot
7,The chef's new fusion dish made a hit among th...,figurative,make hit,Zero-shot
8,Their innovative advertising campaign made a h...,figurative,make hit,Zero-shot
9,His revolutionary approach to teaching made a ...,figurative,make hit,Zero-shot


In [2430]:
df_structured = pd.concat([df_structured, new_df], ignore_index=True)
df_structured

,submission,category,idiom,type
0,The new musician hoped his fresh take on jazz ...,figurative,make hit,Zero-shot
1,Trying innovative marketing strategies can som...,figurative,make hit,Zero-shot
2,"When they launched their new software update, ...",figurative,make hit,Zero-shot
3,The novelist's latest thriller made a hit and ...,figurative,make hit,Zero-shot
4,She made a hit at the conference by introducin...,figurative,make hit,Zero-shot
5,The mobile app made a hit in the tech communit...,figurative,make hit,Zero-shot
6,"Her speech at the gala made a hit, leaving the...",figurative,make hit,Zero-shot
7,The chef's new fusion dish made a hit among th...,figurative,make hit,Zero-shot
8,Their innovative advertising campaign made a h...,figurative,make hit,Zero-shot
9,His revolutionary approach to teaching made a ...,figurative,make hit,Zero-shot


In [2431]:
df_structured

,submission,category,idiom,type
0,The new musician hoped his fresh take on jazz ...,figurative,make hit,Zero-shot
1,Trying innovative marketing strategies can som...,figurative,make hit,Zero-shot
2,"When they launched their new software update, ...",figurative,make hit,Zero-shot
3,The novelist's latest thriller made a hit and ...,figurative,make hit,Zero-shot
4,She made a hit at the conference by introducin...,figurative,make hit,Zero-shot
5,The mobile app made a hit in the tech communit...,figurative,make hit,Zero-shot
6,"Her speech at the gala made a hit, leaving the...",figurative,make hit,Zero-shot
7,The chef's new fusion dish made a hit among th...,figurative,make hit,Zero-shot
8,Their innovative advertising campaign made a h...,figurative,make hit,Zero-shot
9,His revolutionary approach to teaching made a ...,figurative,make hit,Zero-shot


**Enhanced-prompting**

In [2432]:
class ChatSession:
    def __init__(self, category, model=model):
        self.model = model
        system_message = f"Since English is your native language, you should use our language in rich and creative ways. Please respond carefully according to the specified request, without altering the root words of the idiom. Also, you should not make sentences using personal names!"
        print(system_message)
        self.messages = [
            {"role": "system", "content": system_message}
        ]

    def get_completion(self, user_prompt):
        # Add the new user message
        self.messages.append({"role": "user", "content": user_prompt})
        
        # Make the API call
        response = client.chat.completions.create(
            model=self.model,
            messages=self.messages,
        )

        # Retrieve the response and append to messages to maintain context
        ai_response = response.choices[0].message.content.strip()
        self.messages.append({"role": "assistant", "content": ai_response})
        response_df = pd.DataFrame(columns=["raw_response", "idiom","type","category"])
        new_entry = {
            "raw_response": ai_response,
            "idiom": idiom,
            "type": type,
            "category":category
        }
        response_df = pd.concat([response_df, pd.DataFrame([new_entry])], ignore_index=True)
        response_df.to_csv('response_data.csv', mode='a', header=False, index=False)
        return ai_response


prompt = f"""
The idiom 'break the ice' can be used in various educational situations, such as starting a new school year, introducing a new topic, engaging shy students, encouraging group discussions, or welcoming new students to the class. Similarly, can you list the situations where the idiom '{idiom}' is used in a {category} sense?
"""


In [2433]:
chat_session = ChatSession(category=category)

Since English is your native language, you should use our language in rich and creative ways. Please respond carefully according to the specified request, without altering the root words of the idiom. Also, you should not make sentences using personal names!


In [2434]:
type = "Enhanced-prompting"
step = 1

In [2435]:
print(prompt)
print("\n")

for i in range(step):
    result = chat_session.get_completion(prompt)
    print(result)


The idiom 'break the ice' can be used in various educational situations, such as starting a new school year, introducing a new topic, engaging shy students, encouraging group discussions, or welcoming new students to the class. Similarly, can you list the situations where the idiom 'make hit' is used in a figurative sense?





Certainly! The idiom "make a hit" can be used in various situations to describe achieving success or being very well received in a particular setting. Here are some scenarios where this idiom might be used figuratively:

1. **In a Business Presentation:** When a new product or marketing strategy is introduced during a meeting and it receives positive feedback from colleagues or stakeholders, one could say it "made a hit."

2. **During a Performance:** Whether in theater, dance, or a musical performance, if the show is exceptionally well-received by the audience, it is said to have "made a hit."

3. **At a Social Gathering:** When someone attends a party or social event and quickly becomes popular or well-liked due to their charm or charisma, they might be described as having "made a hit" with the crowd.

4. **In the Film or Music Industry:** When a new movie or song garners widespread acclaim and popularity, it is often referred to as having "made a hit."

5. **In Publishing:** If a bo

In [2436]:
prompt = f"""
For each of the scenarios listed in the previous message where the idiom '{idiom}' is used in the {category} sense, create four different sentences. The first sentence should be long and detailed, the second in the form of a question, the third in a negative construction, and the fourth should have other words interspersed among the words that make up the idiom. Please do not alter the root words of the idiom. You should create four different sentences for each category. Return only these sentences in your response.
"""

In [2437]:
print(prompt)
responses = []
for i in range(step):
    result = chat_session.get_completion(prompt)
    responses.append(result)


For each of the scenarios listed in the previous message where the idiom 'make hit' is used in the figurative sense, create four different sentences. The first sentence should be long and detailed, the second in the form of a question, the third in a negative construction, and the fourth should have other words interspersed among the words that make up the idiom. Please do not alter the root words of the idiom. You should create four different sentences for each category. Return only these sentences in your response.



In [2438]:
responses

["1. **In a Business Presentation:**\n   - With a meticulously prepared slide deck that combined insightful analyses with innovative ideas to improve productivity, he unquestionably made a hit at the annual corporate meeting.\n   - Did her new marketing pitch make a hit with the board of directors?\n   - Despite her efforts, her proposal sadly did not make a hit during the recent startup pitch event.\n   - She made an unexpected hit with her proposal at the conference.\n\n2. **During a Performance:**\n   - The young actress, playing the lead role with an extraordinary blend of emotion and elegance, made a hit on her opening night, receiving a standing ovation from the audience.\n   - Did the band's new song make a hit at the live concert last night?\n   - The play didn't make a hit as expected, with only a few scattered claps at the end.\n   - His performance on stage effortlessly made the audience's favorite hit of the evening.\n\n3. **At a Social Gathering:**\n   - At the high-profil

In [2439]:
def parse_and_structure_data(raw_responses_list, idiom, category, response_type):
    structured_data = []
    # Iterate through each block of responses
    for block in raw_responses_list:
        # Split each block into individual lines
        lines = block.split('\n')
        # Skip the header line (assumes the header is the first line)
        for line in lines[1:]:  # this skips the header which is lines[0]
            # Check if the line contains an actual sentence
            if line.strip() and '-' in line:
                # Remove the bullet point and any leading/trailing spaces
                sentence = line.split('-', 1)[-1].strip()
                # Append the structured tuple
                structured_data.append((sentence, category, idiom, response_type))
    return structured_data

In [2440]:
def parse_numbered_sentences(raw_texts):
    # Initialize the list to store extracted sentences
    extracted_sentences = []
    
    # Iterate through each text block in the list
    for raw_text in raw_texts:
        # Split the input text into lines
        lines = raw_text.split('\n')
        
        # Iterate through each line in the input text
        for line in lines:
            # Strip any leading/trailing whitespace from the line
            line = line.strip()
            
            # Check if the line starts with a numeric prefix followed by a dot and space (e.g., '1. ')
            if line.startswith(('1. ', '2. ', '3. ', '4. ')):
                # Extract the sentence after the numeric prefix
                sentence = line.split('. ', 1)[-1]
                # Append the extracted sentence to the list
                extracted_sentences.append((sentence, category, idiom, type))
    
    return extracted_sentences

In [2441]:
parsed_data = parse_and_structure_data(responses, idiom, category, type)
parsed_data

[('With a meticulously prepared slide deck that combined insightful analyses with innovative ideas to improve productivity, he unquestionably made a hit at the annual corporate meeting.',
  'figurative',
  'make hit',
  'Enhanced-prompting'),
 ('Did her new marketing pitch make a hit with the board of directors?',
  'figurative',
  'make hit',
  'Enhanced-prompting'),
 ('Despite her efforts, her proposal sadly did not make a hit during the recent startup pitch event.',
  'figurative',
  'make hit',
  'Enhanced-prompting'),
 ('She made an unexpected hit with her proposal at the conference.',
  'figurative',
  'make hit',
  'Enhanced-prompting'),
 ('The young actress, playing the lead role with an extraordinary blend of emotion and elegance, made a hit on her opening night, receiving a standing ovation from the audience.',
  'figurative',
  'make hit',
  'Enhanced-prompting'),
 ("Did the band's new song make a hit at the live concert last night?",
  'figurative',
  'make hit',
  'Enhance

In [2442]:
new_df= pd.DataFrame(parsed_data, columns=["submission", "category", "idiom", "type"])
new_df

,submission,category,idiom,type
0,With a meticulously prepared slide deck that c...,figurative,make hit,Enhanced-prompting
1,Did her new marketing pitch make a hit with th...,figurative,make hit,Enhanced-prompting
2,"Despite her efforts, her proposal sadly did no...",figurative,make hit,Enhanced-prompting
3,She made an unexpected hit with her proposal a...,figurative,make hit,Enhanced-prompting
4,"The young actress, playing the lead role with ...",figurative,make hit,Enhanced-prompting
5,Did the band's new song make a hit at the live...,figurative,make hit,Enhanced-prompting
6,"The play didn't make a hit as expected, with o...",figurative,make hit,Enhanced-prompting
7,His performance on stage effortlessly made the...,figurative,make hit,Enhanced-prompting
8,"At the high-profile gala, wearing a stunningly...",figurative,make hit,Enhanced-prompting
9,Did he make a hit at your sister’s wedding rec...,figurative,make hit,Enhanced-prompting


In [2443]:
df_structured = pd.concat([df_structured, new_df], ignore_index=True)
df_structured


,submission,category,idiom,type
0,The new musician hoped his fresh take on jazz ...,figurative,make hit,Zero-shot
1,Trying innovative marketing strategies can som...,figurative,make hit,Zero-shot
2,"When they launched their new software update, ...",figurative,make hit,Zero-shot
3,The novelist's latest thriller made a hit and ...,figurative,make hit,Zero-shot
4,She made a hit at the conference by introducin...,figurative,make hit,Zero-shot
...,...,...,...,...
79,His book surprisingly made an unexpected hit i...,figurative,make hit,Enhanced-prompting
80,"The innovative chef, who introduced a fusion d...",figurative,make hit,Enhanced-prompting
81,Did her new vegan dessert make a hit at the ho...,figurative,make hit,Enhanced-prompting
82,"The new menu item didn't make a hit, remaining...",figurative,make hit,Enhanced-prompting


In [2444]:
class ChatSession:
    def __init__(self, category, model=model):
        self.model = model
        system_message = f"Since English is your native language, you should use our language in rich and creative ways. Please avoid repeating your sentences and craft sentences using idioms correctly and with their {category} meanings, without altering the root words of the idiom. You should also avoid constructing sentences using personal names!"
        print(system_message)
        self.messages = [
            {"role": "system", "content": system_message}
        ]

    def get_completion(self, user_prompt):
        # Add the new user message
        self.messages.append({"role": "user", "content": user_prompt})
        
        # Make the API call
        response = client.chat.completions.create(
            model=self.model,
            messages=self.messages,
        )

        # Retrieve the response and append to messages to maintain context
        ai_response = response.choices[0].message.content.strip()
        self.messages.append({"role": "assistant", "content": ai_response})
        response_df = pd.DataFrame(columns=["raw_response", "idiom","type","category"])
        new_entry = {
            "raw_response": ai_response,
            "idiom": idiom,
            "type": type,
            "category":category
        }
        response_df = pd.concat([response_df, pd.DataFrame([new_entry])], ignore_index=True)
        response_df.to_csv('response_data.csv', mode='a', header=False, index=False)
        return ai_response

In [2445]:
chat_session = ChatSession(category=category)

Since English is your native language, you should use our language in rich and creative ways. Please avoid repeating your sentences and craft sentences using idioms correctly and with their figurative meanings, without altering the root words of the idiom. You should also avoid constructing sentences using personal names!


In [2454]:
type = "Enhanced-prompting"
step = 2

In [2455]:
prompt = f"""
Using the idiom '{idiom}' in the {category} sense, create five new sentences with different sentence structures. Use the idiom in various parts of the sentences. Please avoid repeating your sentences.
"""
print(prompt)
print("\n")
responses = []
for i in range(step):
    result = chat_session.get_completion(prompt)
    responses.append(result)


Using the idiom 'make hit' in the figurative sense, create five new sentences with different sentence structures. Use the idiom in various parts of the sentences. Please avoid repeating your sentences.





In [2456]:
responses

['1. Expecting to make a hit, the jazz band debuted their catchy new tune to a packed house.\n2. To make a hit on social media, Mark curated a series of stunning nature photographs.\n3. Did his unique approach make a hit, or did it fall flat among the critics?\n4. The young inventor aimed to make a hit with her eco-friendly gadget at the science fair.\n5. By redesigning their app to make it more user-friendly, the developers hoped to make a hit in the competitive market.',
 "1. The director's new experimental film managed to make a hit among indie enthusiasts.\n2. The chef hopes his new fusion menu will make a hit among food critics.\n3. Will the refurbished vintage car make a hit at the classic auto show this weekend?\n4. Making a hit at the international conference could significantly benefit her career.\n5. She decided to wear her most glamorous dress in an effort to make a hit at the gala."]

In [2457]:
def parse_and_structure_data(raw_responses_list, idiom, category, response_type):
    # Initialize the list to store structured data
    structured_data = []
    # Iterate through each block of responses
    for block in raw_responses_list:
        # Split each block into individual sentences
        sentences = block.split('\n')
        # Remove the numbering from each sentence and strip leading/trailing whitespace
        clean_sentences = [sentence.split('. ', 1)[-1].strip() for sentence in sentences]
        # For each sentence, append a tuple with the structured data
        for sentence in clean_sentences:
            structured_data.append((sentence, category, idiom, response_type))
    return structured_data

In [2458]:
structured_data = parse_and_structure_data(responses, idiom, category, type)

new_df= pd.DataFrame(structured_data, columns=["submission", "category", "idiom", "type"])
new_df

,submission,category,idiom,type
0,"Expecting to make a hit, the jazz band debuted...",figurative,make hit,Enhanced-prompting
1,"To make a hit on social media, Mark curated a ...",figurative,make hit,Enhanced-prompting
2,"Did his unique approach make a hit, or did it ...",figurative,make hit,Enhanced-prompting
3,The young inventor aimed to make a hit with he...,figurative,make hit,Enhanced-prompting
4,By redesigning their app to make it more user-...,figurative,make hit,Enhanced-prompting
5,The director's new experimental film managed t...,figurative,make hit,Enhanced-prompting
6,The chef hopes his new fusion menu will make a...,figurative,make hit,Enhanced-prompting
7,Will the refurbished vintage car make a hit at...,figurative,make hit,Enhanced-prompting
8,Making a hit at the international conference c...,figurative,make hit,Enhanced-prompting
9,She decided to wear her most glamorous dress i...,figurative,make hit,Enhanced-prompting


In [2459]:
new_df = new_df[new_df['submission']!=""]
new_df.reset_index(drop=True,inplace=True)
new_df


,submission,category,idiom,type
0,"Expecting to make a hit, the jazz band debuted...",figurative,make hit,Enhanced-prompting
1,"To make a hit on social media, Mark curated a ...",figurative,make hit,Enhanced-prompting
2,"Did his unique approach make a hit, or did it ...",figurative,make hit,Enhanced-prompting
3,The young inventor aimed to make a hit with he...,figurative,make hit,Enhanced-prompting
4,By redesigning their app to make it more user-...,figurative,make hit,Enhanced-prompting
5,The director's new experimental film managed t...,figurative,make hit,Enhanced-prompting
6,The chef hopes his new fusion menu will make a...,figurative,make hit,Enhanced-prompting
7,Will the refurbished vintage car make a hit at...,figurative,make hit,Enhanced-prompting
8,Making a hit at the international conference c...,figurative,make hit,Enhanced-prompting
9,She decided to wear her most glamorous dress i...,figurative,make hit,Enhanced-prompting


In [2460]:
df_structured = pd.concat([df_structured, new_df], ignore_index=True)

df_structured

,submission,category,idiom,type
0,The new musician hoped his fresh take on jazz ...,figurative,make hit,Zero-shot
1,Trying innovative marketing strategies can som...,figurative,make hit,Zero-shot
2,"When they launched their new software update, ...",figurative,make hit,Zero-shot
3,The novelist's latest thriller made a hit and ...,figurative,make hit,Zero-shot
4,She made a hit at the conference by introducin...,figurative,make hit,Zero-shot
...,...,...,...,...
95,The director's new experimental film managed t...,figurative,make hit,Enhanced-prompting
96,The chef hopes his new fusion menu will make a...,figurative,make hit,Enhanced-prompting
97,Will the refurbished vintage car make a hit at...,figurative,make hit,Enhanced-prompting
98,Making a hit at the international conference c...,figurative,make hit,Enhanced-prompting


In [2461]:
df_structured

,submission,category,idiom,type
0,The new musician hoped his fresh take on jazz ...,figurative,make hit,Zero-shot
1,Trying innovative marketing strategies can som...,figurative,make hit,Zero-shot
2,"When they launched their new software update, ...",figurative,make hit,Zero-shot
3,The novelist's latest thriller made a hit and ...,figurative,make hit,Zero-shot
4,She made a hit at the conference by introducin...,figurative,make hit,Zero-shot
...,...,...,...,...
95,The director's new experimental film managed t...,figurative,make hit,Enhanced-prompting
96,The chef hopes his new fusion menu will make a...,figurative,make hit,Enhanced-prompting
97,Will the refurbished vintage car make a hit at...,figurative,make hit,Enhanced-prompting
98,Making a hit at the international conference c...,figurative,make hit,Enhanced-prompting


In [2462]:
# Define the path for the CSV file
file_path = 'gpt4_en_final_mecaz.csv'

# Check if the file exists
if os.path.exists(file_path):
    # Append without writing headers
    df_structured.to_csv(file_path, mode='a', index=False, header=False)
else:
    # Create a new file and write the DataFrame with headers
    df_structured.to_csv(file_path, mode='w', index=False, header=True)